In [21]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.19 which is incompatible.
autogluon-multimodal 1.4.0 requires jsonschema<4.24,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.55.2 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0,

In [2]:
from utils import setup_cognito_user_pool

# setup cognito for authentication

In [3]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(cognito_config)

Setting up Amazon Cognito user pool...
Pool id: us-west-2_jQsMayVcr
Discovery URL: https://cognito-idp.us-west-2.amazonaws.com/us-west-2_jQsMayVcr/.well-known/openid-configuration
Client ID: 552tnn4svdahqmk0e9utkbreaa
Bearer Token: eyJraWQiOiJuZGZwVjhHc0VVVkZHNWJrU1E0XC9hWHJBVngyUTREeEtYbDN6Rk5EY002Zz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI3ODkxZDMyMC03MGMxLTcwZjUtYWU3Ny1kMDQ4NGVlMTAzNDEiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtd2VzdC0yLmFtYXpvbmF3cy5jb21cL3VzLXdlc3QtMl9qUXNNYXlWY3IiLCJjbGllbnRfaWQiOiI1NTJ0bm40c3ZkYWhxbWswZTl1dGticmVhYSIsIm9yaWdpbl9qdGkiOiJiYmVjM2ExOS1jMTViLTQ1ZjMtYmE0Zi0yZjhjZmYyZTg1MzkiLCJldmVudF9pZCI6IjY5NThjY2ViLWU1NTktNDc2OC1iMDI2LWU3MWRkYzViYTNjZCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NTYzNzMwNjksImV4cCI6MTc1NjM3NjY2OSwiaWF0IjoxNzU2MzczMDY5LCJqdGkiOiJhYWY5OGQwZS02MDQ2LTQ5ZDQtYjZlOC1kZDMzZjAwZjBjYmQiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.QDyOAvg2sQLS0RbJB5R565ZJ52pMhrBFrYrR4tccnJ9aljm_yQ3a6dUqOfogQpp71w5o3_2K2iH

# config agentcore runtime hosting MCP server

In [5]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time
import os

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['awslabs/aws_support_mcp_server/server.py', 'requirements.txt']
for file in required_files:
    print({file})
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="awslabs/aws_support_mcp_server/server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="awssupportmcpserver"
)
print("Configuration completed ✓")

Entrypoint parsed: file=/home/sagemaker-user/AgentCore-Prod/AWS-Support-MCP/MCP/awslabs/aws_support_mcp_server/server.py, bedrock_agentcore_name=server
Configuring BedrockAgentCore agent: awssupportmcpserver


Using AWS region: us-west-2
{'awslabs/aws_support_mcp_server/server.py'}
{'requirements.txt'}
All required files found ✓
Configuring AgentCore Runtime...


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /home/sagemaker-user/AgentCore-Prod/AWS-Support-MCP/MCP/Dockerfile
Generated .dockerignore: /home/sagemaker-user/AgentCore-Prod/AWS-Support-MCP/MCP/.dockerignore
Setting 'awssupportmcpserver' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/AgentCore-Prod/AWS-Support-MCP/MCP/.bedrock_agentcore.yaml


Configuration completed ✓


# create agentcore runtime

In [6]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'awssupportmcpserver' to account 887221633712 (us-west-2)
Starting CodeBuild ARM64 deployment for agent 'awssupportmcpserver' to account 887221633712 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: awssupportmcpserver


Launching MCP server to AgentCore Runtime...
This may take several minutes...
Repository doesn't exist, creating new ECR repository: bedrock-agentcore-awssupportmcpserver


✅ ECR repository available: 887221633712.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-awssupportmcpserver
Getting or creating execution role for agent: awssupportmcpserver
Using AWS region: us-west-2, account ID: 887221633712
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-de689bc2c1
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-de689bc2c1
Starting execution role creation process for agent: awssupportmcpserver
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-west-2-de689bc2c1
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-de689bc2c1
✓ Role created: arn:aws:iam::887221633712:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-de689bc2c1
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-awssupportmcpserver
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::887221633712:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-de689bc2c1
Preparing CodeBuild

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:us-west-2:887221633712:runtime/awssupportmcpserver-7ESps168I4
Agent ID: awssupportmcpserver-7ESps168I4


# check agentcore runtime status

In [7]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

Checking AgentCore Runtime status...


Retrieved Bedrock AgentCore status for: awssupportmcpserver


Initial status: READY
✓ AgentCore Runtime is READY!
Final status: READY


# upload data to secret and ssm

In [14]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='support_mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='support_mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/support_mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Cognito credentials stored in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:us-west-2:887221633712:runtime/awssupportmcpserver-7ESps168I4


# List MCP tools

In [15]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/support_mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='support_mcp_server/cognito/credentials')
        print(response)
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Writing my_mcp_client_remote.py


In [16]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: us-west-2
Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-west-2:887221633712:runtime/awssupportmcpserver-7ESps168I4
{'ARN': 'arn:aws:secretsmanager:us-west-2:887221633712:secret:support_mcp_server/cognito/credentials-hAzPWS', 'Name': 'support_mcp_server/cognito/credentials', 'VersionId': '039db42e-8fb0-4255-8987-85dec5af139d', 'SecretString': '{"pool_id": "us-west-2_jQsMayVcr", "client_id": "552tnn4svdahqmk0e9utkbreaa", "bearer_token": "eyJraWQiOiJuZGZwVjhHc0VVVkZHNWJrU1E0XC9hWHJBVngyUTREeEtYbDN6Rk5EY002Zz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI3ODkxZDMyMC03MGMxLTcwZjUtYWU3Ny1kMDQ4NGVlMTAzNDEiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtd2VzdC0yLmFtYXpvbmF3cy5jb21cL3VzLXdlc3QtMl9qUXNNYXlWY3IiLCJjbGllbnRfaWQiOiI1NTJ0bm40c3ZkYWhxbWswZTl1dGticmVhYSIsIm9yaWdpbl9qdGkiOiJiYmVjM2ExOS1jMTViLTQ1ZjMtYmE0Zi0yZjhjZmYyZTg1MzkiLCJldmVudF9pZCI6IjY5NThjY2ViLWU1NTktNDc2OC1iMDI2LWU3MWRkYzViYTNjZCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbm

# create local agent which is able to invoke MCP Server
# grant AWSSupportAccess to agentcore runtime 

In [22]:
%%writefile agent_invoke_mcp_tools_final.py
import asyncio
import boto3
import json
import sys
from strands import Agent
from boto3.session import Session
from datetime import timedelta
from strands.tools.mcp import MCPClient

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from datetime import datetime, timedelta, timezone

def get_system_prompt() -> str:
    """
    Return the full system prompt for the AWS Support Case Management Agent,
    including dynamic current Beijing Time (UTC+8).
    """
    # 获取当前北京时间
    beijing_tz = timezone(timedelta(hours=8))
    now_bj = datetime.now(beijing_tz)
    current_time_str = now_bj.strftime("%Y-%m-%d %H:%M:%S %Z")

    # 定义 system prompt
    system_prompt = f"""
# AWS Support Case Management Agent - Cross-border E-commerce IT Support

## Current Reference Time
The current Beijing Time (UTC+8) is: **{current_time_str}**.
- When the user mentions relative time ranges (e.g., "last month", "yesterday", "过去一周"), always calculate them based on this reference time.
- Always assume the user's timezone is Beijing Time (UTC+8) unless explicitly stated otherwise.

## Identity & Role
You are an AWS Support Case Management Agent specialized in serving cross-border e-commerce IT teams. You have deep expertise in AWS services commonly used in e-commerce operations and understand the critical nature of e-commerce business continuity.

## Language Preference
- When interacting with the customer, always **prefer Chinese** for explanations and responses, unless the customer explicitly requests English.

## Core Capabilities
You can create, query, analyze, and provide strategic insights on AWS Support cases using the AWS Support MCP server. Your analysis focuses on business impact, operational efficiency, and proactive problem prevention for large-scale e-commerce operations.

## Target User Profile
- **Primary Users**: IT personnel at large-scale cross-border e-commerce companies
- **Business Context**: High-volume, time-sensitive e-commerce operations with global reach
- **Critical Requirements**: 24/7 availability, data security, compliance, and minimal downtime tolerance

## Key Responsibilities
1. **Case Creation & Management**: Create well-structured support cases with appropriate severity and business impact.
2. **Intelligent Case Analysis**: Summarize, classify, and analyze cases by severity, service, and category.
3. **Strategic Insights**: Provide best practices, preventive actions, and business impact assessments.
4. **Escalation Guidance**: Identify critical cases that require urgent action or executive visibility.

## E-commerce Specific Considerations
- Prioritize cases affecting **payment systems, website performance, databases, and security**.
- Pay special attention during **peak shopping seasons** and **flash sale events**.
- Map case severity to business impact (Critical = revenue-stopping, High = degraded CX, etc.).

## Output Guidelines
- Always provide summaries with **executive overviews** + **detailed breakdowns**.
- Highlight **critical cases** and **patterns**.
- Provide **clear, actionable recommendations**.
- Keep responses **concise, structured, and business-focused**.
- Always connect technical issues to **business impact**.

## Security & Compliance
- Always consider PCI DSS, GDPR, and cross-border data compliance in analysis.
- Treat payment and security-related cases as **highest priority**.

Remember: Your primary goal is to help e-commerce IT teams maintain robust, secure, and high-performing AWS infrastructure that supports their business objectives while minimizing operational risks and maximizing customer satisfaction.
"""
    return system_prompt



async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/support_mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='support_mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    def create_streamable_http_transport():
        return streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False)
    streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

    system_prompt = get_system_prompt()
    
    with streamable_http_mcp_client:
        tools = streamable_http_mcp_client.list_tools_sync()
        print(f"tools list: {tools}")
        agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            tools=tools,system_prompt=system_prompt)
        response = str(agent("show me the case which severity is General guidance,and give me insight?"))

if __name__ == "__main__":
    asyncio.run(main())

Overwriting agent_invoke_mcp_tools_final.py


In [24]:
print("Testing MCP tool invocation from Agent...")
print("=" * 50)
!python agent_invoke_mcp_tools_final.py

Testing MCP tool invocation from Agent...
Using AWS region: us-west-2
Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-west-2:887221633712:runtime/awssupportmcpserver-7ESps168I4
✓ Retrieved bearer token from Secrets Manager

Connecting to: https://bedrock-agentcore.us-west-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-west-2%3A887221633712%3Aruntime%2Fawssupportmcpserver-7ESps168I4/invocations?qualifier=DEFAULT
tools list: [<strands.tools.mcp.mcp_agent_tool.MCPAgentTool object at 0x7febd734e210>, <strands.tools.mcp.mcp_agent_tool.MCPAgentTool object at 0x7febd64e1160>, <strands.tools.mcp.mcp_agent_tool.MCPAgentTool object at 0x7febd64e02c0>, <strands.tools.mcp.mcp_agent_tool.MCPAgentTool object at 0x7febd64e0860>, <strands.tools.mcp.mcp_agent_tool.MCPAgentTool object at 0x7febd64e1a00>, <strands.tools.mcp.mcp_agent_tool.MCPAgentTool object at 0x7febd64e19d0>]
我需要为您查询严重程度为"低级别"(General guidance)的AWS支持案例。让我来帮您获取这些信息并提供相关分析。
Tool #1: describe_severity_levels
我了解了支持案例的严重程度级别